<a href="https://colab.research.google.com/github/alan-w25/ese4380-project/blob/main/cir_sims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np


def simulate_cir_paths(kappa, theta, sigma, v0, T, n_steps, n_paths, seed=None):
    """
    Exact CIR simulation via noncentral chi-square transition.
    Returns V with shape (n_paths, n_steps+1), including t=0.
    """
    rng = np.random.default_rng(seed)
    dt = T / n_steps

    # deterministic case (sigma=0)
    if sigma == 0.0:
        times = np.linspace(0, T, n_steps + 1)
        V_det = theta + (v0 - theta) * np.exp(-kappa * times)
        return np.tile(V_det, (n_paths, 1))

    exp_kdt = np.exp(-kappa * dt)
    one_minus_exp = -np.expm1(-kappa * dt)          # 1 - e^{-kappa dt}, stable
    nu = 4.0 * kappa * theta / (sigma**2)           # dof
    c  = (sigma**2 * one_minus_exp) / (4.0 * kappa) # scale

    V = np.empty((n_paths, n_steps + 1), dtype=np.float64)
    V[:, 0] = v0

    for i in range(n_steps):
        v_t = V[:, i]
        lam = (4.0 * kappa * exp_kdt / (sigma**2 * one_minus_exp)) * v_t
        X = rng.noncentral_chisquare(df=nu, nonc=lam, size=n_paths)
        V[:, i + 1] = c * X

    return V


In [3]:
def power_mean_moments(V, K=10):
    """
    For each time t (columns of V), compute the length-K vector:
        M_k(t) = ( (1/S) * sum_{s=1}^S V[s, t]^k )^(1/k),   k=1..K
    where S = number of paths (rows of V).

    Parameters
    ----------
    V : ndarray of shape (n_paths, T_plus)
        Simulated paths (each column is a time step t).
    K : int
        Highest order.

    Returns
    -------
    M : ndarray of shape (T_plus, K)
        Row t contains [M_1(t), M_2(t), ..., M_K(t)].
    """
    n_paths, T_plus = V.shape
    M = np.empty((T_plus, K), dtype=np.float64)

    # iterative powers
    pow_k = V.copy()
    for k in range(1, K + 1):
        mean_pow = pow_k.mean(axis=0)
        M[:, k - 1] = mean_pow if k == 1 else mean_pow**(1.0 / k)
        pow_k *= V

    return M


In [7]:

from google.colab import files

def run_many_sims_no_chunk_then_moments(
    kappa, theta, sigma, v0, T, n_steps,
    paths_per_run, n_runs, K=10, base_seed=123
):
    """
    Repeats full CIR simulations n_runs times (no chunking in each run),
    stacks all paths, then computes power-mean moments once.

    Returns:
      M : (T_plus, K)   power-mean moments using ALL paths across runs
    """
    all_V = []
    for r in range(n_runs):
        # simple decorrelation
        seed = None if base_seed is None else (base_seed + 9973 * r)
        V = simulate_cir_paths(
            kappa, theta, sigma, v0, T, n_steps, paths_per_run, seed=seed
        )
        all_V.append(V)

    V_all = np.vstack(all_V)
    M = power_mean_moments(V_all, K=K)
    np.savetxt("M.csv", M, delimiter=",", fmt="%.8f")
    files.download("M.csv")
    return M, V_all


In [8]:
run_many_sims_no_chunk_then_moments(kappa = 1.5, theta = 0.04, sigma = 0.25, v0 = 0.04, T = 1, n_steps = 1000, paths_per_run = 50000, n_runs = 1, K=10, base_seed=123)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

(array([[0.04      , 0.04      , 0.04      , ..., 0.04      , 0.04      ,
         0.04      ],
        [0.04000029, 0.04003142, 0.04006254, ..., 0.04021788, 0.0402489 ,
         0.04027991],
        [0.04000285, 0.04006539, 0.04012787, ..., 0.04043941, 0.04050153,
         0.04056359],
        ...,
        [0.04004671, 0.04889413, 0.05736267, ..., 0.09719356, 0.10462297,
         0.11178572],
        [0.04004445, 0.04889598, 0.05737542, ..., 0.09743264, 0.10496224,
         0.11225187],
        [0.04003119, 0.04887881, 0.05735099, ..., 0.09729064, 0.10475204,
         0.1119501 ]]),
 array([[0.04      , 0.04201089, 0.04429141, ..., 0.0667528 , 0.0685916 ,
         0.07224646],
        [0.04      , 0.04089735, 0.03981059, ..., 0.00763969, 0.00781645,
         0.00807986],
        [0.04      , 0.03945874, 0.041279  , ..., 0.01151014, 0.01167641,
         0.01159576],
        ...,
        [0.04      , 0.03811807, 0.03876591, ..., 0.01853682, 0.01814416,
         0.01945801],
        [0.0